In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import datasets as ds


## The dataset

### Question 1
How many splits does the dataset has?

In [6]:
splits = ds.get_dataset_split_names('imdb')
print(splits)
print(f'Number of splits: {len(splits)}')

['train', 'test', 'unsupervised']
Number of splits: 3


There are 3 splits in the IMDB dataset.

### Question 2
How big are these splits?

In [7]:
dataset_0 = ds.load_dataset('imdb', split=splits[0])
dataset_1 = ds.load_dataset('imdb', split=splits[1])
dataset_2 = ds.load_dataset('imdb', split=splits[2])

Found cached dataset imdb (/home/bastien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/home/bastien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Found cached dataset imdb (/home/bastien/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


In [8]:
print(f'{splits[0]} split size : {dataset_0.num_rows}')
print(f'{splits[1]} split size : {dataset_1.num_rows}')
print(f'{splits[2]} split size : {dataset_2.num_rows}')

train split size : 25000
test split size : 25000
unsupervised split size : 50000


### Question 3
What is the proportion of each class on the supervised splits?

In [25]:
# Dataset 0: Train
train_data_frame = dataset_0.to_pandas()
print("Train")
print(train_data_frame.where(train_data_frame['label'] == 0).count())
print(train_data_frame.where(train_data_frame['label'] == 1).count())

print("\n")
# Dataset 1: Test
print("Test")
test_data_frame = dataset_1.to_pandas()
print(test_data_frame.where(test_data_frame['label'] == 0).count())
print(test_data_frame.where(test_data_frame['label'] == 1).count())


Train
text     12500
label    12500
dtype: int64
text     12500
label    12500
dtype: int64


Test
text     12500
label    12500
dtype: int64
text     12500
label    12500
dtype: int64


Hence, each class represents 50% of the supervised dataset (both in train and test samples).

## Naive Bayes classifier 

### Question 1
Create an adapted processing function which lower case the text and replace punctuations with text:

In [37]:
import string
import re

def replace_punctuation(text: str, replace: str)-> str:
  return re.sub('[%s]' % re.escape(string.punctuation), replace, text)

def remove_multiple_space(text : str) -> str:
  return re.sub(' +', ' ', text)

def text_processing(text: str) -> str:
    result_text = replace_punctuation(text,' ') #text.translate(str.maketrans('','',string.punctuation))
    result_text = result_text.lower()
    result_text = result_text.strip()
    result_text = remove_multiple_space(result_text)
    return result_text

#tiny test
text_processing("Hello, ,,,World!::")

'hello world'

Now let's apply `text_processing` function on our dataframe.

In [39]:
train_data_frame.text.apply(text_processing)[1]

'i am curious yellow is a risible and pretentious steaming pile it doesn t matter what one s political views are because this film can hardly be taken seriously on any level as for the claim that frontal male nudity is an automatic nc 17 that isn t true i ve seen r rated films with male nudity granted they only offer some fleeting views but where are the r rated films with gaping vulvas and flapping labia nowhere because they don t exist the same goes for those crappy cable shows schlongs swinging in the breeze but not a clitoris in sight and those pretentious indie movies like the brown bunny in which we re treated to the site of vincent gallo s throbbing johnson but not a trace of pink visible on chloe sevigny before crying or implying double standard in matters of nudity the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women there are no genitals on display when actresses appears nude and the same cannot be said for a man in 